In [1]:
import random
import math
%run utils.ipynb

In [2]:
class Individual:
    def __init__(self, words, alphabets):
        self.code = generate_random_string_from_alphabets(alphabets)
        self.calc_fitness(words)

    def calc_fitness(self, words):
        self.fitness = max_hamming_distance(self.code, words)

## Tournament Selection

- randomly select `k` number of participants from population
- the best of those participants is the winner and is selected for crossover
- since the fitness is `max hamming distance`, the best individual is the one with the smallest fitness

In [3]:
def _selection(population, k):
    num_participants = min(k, len(population))
    participants = random.sample(population, num_participants)
    return min(participants, key=lambda x: x.fitness)

## Uniform Crossover

- for each position flip a coin (50-50) to determine if the child gets character from parent1 or parent2
- changes child in-place

In [4]:
def _crossover(child, parent1, parent2):
    child.code = "".join([parent1.code[i] if random.random() < 0.5 else parent2.code[i] for i in range(len(parent1.code))])

## Mutation

- iterate over child's code and with probability of `prob` swap each character with the random one from its columns alphabet

In [5]:
def _mutation(child, alphabets, prob):
    code_list = list(child.code)
    for i in range(len(code_list)):
        if random.random() <= prob:
            code_list[i] = random.sample(alphabets[i], 1)[0]
    child.code = "".join(code_list)

In [6]:
def ga_hybrid(words):
    m = len(words[0])
    population_size = int(5 * math.log2(len(words)) + 10 * math.log2(m))
    generations = int(5 * math.log2(m))
    mutation_prob = 0.03
    tournament_size = 3
    elitism_count = max(1, int(0.03 * population_size))
    
    alphabets = get_all_alphabets(words)

    population = [Individual(words, alphabets) for _ in range(population_size)]
    best = min(population, key=lambda x: x.fitness)

    for gen in range(generations):
        population = sorted(population, key=lambda x: x.fitness)
        new_population = population[:elitism_count]
        while len(new_population) < population_size:
            parent1 = _selection(population, tournament_size)
            parent2 = _selection(population, tournament_size)

            child = Individual(words, alphabets)
            _crossover(child, parent1, parent2)
            _mutation(child, alphabets, mutation_prob)
            child.calc_fitness(words)

            new_population.append(child)

        population = new_population

        current_best = min(population, key=lambda x: x.fitness)
        if current_best.fitness < best.fitness:
            best = current_best

    return best.fitness, best.code